In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import datetime
import gc
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error


import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb

%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/pre-processing/data_training.csv
/kaggle/input/pre-processing/__results__.html
/kaggle/input/pre-processing/test_data.csv
/kaggle/input/pre-processing/__notebook__.ipynb
/kaggle/input/pre-processing/__output__.json
/kaggle/input/pre-processing/custom.css
/kaggle/input/pre-processing/__results___files/__results___65_0.png
/kaggle/input/pre-processing/__results___files/__results___8_0.png
/kaggle/input/pre-processing/__results___files/__results___9_0.png
/kaggle/input/pre-processing/__results___files/__results___60_0.png
/kaggle/input/pre-processing/__results___files/__results___35_0.png
/kaggle/input/pre-processing/__results___files/__results___63_0.png
/kaggle/input/pre-processing/__results___files/__results___58_0.png
/kaggle/input/pre-processing/__results___files/__results___54_0.png
/kaggle/input/pre-processing/__results___files/__results___64_0.png
/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

In [4]:

print('-' * 80)
print('train')
training = import_data('/kaggle/input/pre-processing/data_training.csv')

print('-' * 80)
print('test')
data_test = import_data('/kaggle/input/pre-processing/test_data.csv')



--------------------------------------------------------------------------------
train
Memory usage of dataframe is 2610.25 MB
Memory usage after optimization is: 844.49 MB
Decreased by 67.6%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 5090.04 MB
Memory usage after optimization is: 1709.94 MB
Decreased by 66.4%


In [5]:
missing_statistics(data_test)

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,row_id,0,41697600,0.00
1,building_id,0,41697600,0.00
2,meter,0,41697600,0.00
3,timestamp,0,41697600,0.00
4,site_id,0,41697600,0.00
5,primary_use,0,41697600,0.00
6,square_feet,0,41697600,0.00
7,air_temperature,221901,41697600,0.53
8,dew_temperature,260799,41697600,0.63
9,precip_depth_1_hr,7801563,41697600,18.71


In [6]:
training.loc[(training['site_id'] == 0) & (training['meter'] == 0), 'meter_reading'] = training[(training['site_id'] == 0) & (training['meter'] == 0)]['meter_reading'] * 0.2931

In [7]:
data_test = data_test.set_index(['site_id','hour','month'])


In [8]:
#filling air temperature
air_temperature_filler = pd.DataFrame(training.groupby(['site_id','hour','month'])['air_temperature'].mean(),columns=["air_temperature"])
data_test.update(air_temperature_filler,overwrite=False)
del air_temperature_filler

In [9]:
#filling due temperature
due_temperature_filler = pd.DataFrame(training.groupby(['site_id','hour','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
data_test.update(due_temperature_filler,overwrite=False)
del due_temperature_filler

In [10]:

# filling sea_level_pressure
sea_level_filler = pd.DataFrame(training.groupby(['site_id','hour','month'])['sea_level_pressure'].mean(),columns=['sea_level_pressure'])
data_test.update(sea_level_filler,overwrite=False)
del sea_level_filler
gc.collect()

0

In [11]:

#filling wind_direction
wind_direction_filler =  pd.DataFrame(training.groupby(['site_id','hour','month'])['wind_direction'].mean(),columns=['wind_direction'])
data_test.update(wind_direction_filler,overwrite=False)
del wind_direction_filler

In [12]:
#filling  wind speed
wind_speed_filler =  pd.DataFrame(training.groupby(['site_id','hour','month'])['wind_speed'].mean(),columns=['wind_speed'])
data_test.update(wind_speed_filler,overwrite=False)
del wind_speed_filler
gc.collect()

0

In [13]:
#filling precip depth
precip_depth_filler = pd.DataFrame(training.groupby(['site_id','hour','month'])['precip_depth_1_hr'].mean(),columns=['precip_depth_1_hr'])
data_test.update(precip_depth_filler,overwrite=False)
del precip_depth_filler
gc.collect()

0

In [14]:
data_test.reset_index(inplace=True)


In [15]:
missing_statistics(data_test)

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,site_id,0,41697600,0.0
1,hour,0,41697600,0.0
2,month,0,41697600,0.0
3,row_id,0,41697600,0.0
4,building_id,0,41697600,0.0
5,meter,0,41697600,0.0
6,timestamp,0,41697600,0.0
7,primary_use,0,41697600,0.0
8,square_feet,0,41697600,0.0
9,air_temperature,0,41697600,0.0


In [16]:
msk = training['week'] % 4 == 0

df_train = training[~msk]
df_val  = training[msk]

In [17]:
df_val.shape[0] #25% of train data for validation


4928764

In [18]:

df_train.shape[0] #70

15196570

In [19]:
data_test.to_csv('data_test.csv', index=False)


In [20]:
df_train.to_csv('df_train.csv', index=False)
df_val.to_csv('df_val.csv', index=False)